In [24]:
import tkinter as tk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np

In [32]:
class GeneticAlgorithmApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Genetic Algorithm App")

        self.population_size = tk.IntVar(value=100)
        self.mutation_rate = tk.DoubleVar(value=0.5)
        self.generation_limit = tk.IntVar(value=100)
        self.gene_encoding = tk.StringVar(value="Binary")

        self.min_mutation_rate = 0.01
        self.mutation_decay_factor = 0.95

        self.function = lambda x1, x2: (x2 - x1**2)**2 + (1 - x1)**2

        self.create_widgets()

        self.population = None
        self.best_solution = None
        self.iteration = 0

        self.create_initial_population()

        self.x1_range = np.linspace(-5, 5, 100)
        self.x2_range = np.linspace(-5, 5, 100)

        self.update_plot()

        self.plot_function()

        self.create_gene_modification_widgets()

    def create_widgets(self):
        tk.Label(self.master, text="Population Size:").grid(row=0, column=0, sticky="e")
        tk.Entry(self.master, textvariable=self.population_size).grid(row=0, column=1)

        tk.Label(self.master, text="Mutation Rate:").grid(row=1, column=0, sticky="e")
        tk.Entry(self.master, textvariable=self.mutation_rate).grid(row=1, column=1)

        tk.Label(self.master, text="Generation Limit:").grid(row=2, column=0, sticky="e")
        tk.Entry(self.master, textvariable=self.generation_limit).grid(row=2, column=1)

        tk.Label(self.master, text="Gene Encoding:").grid(row=3, column=0, sticky="e")
        tk.OptionMenu(self.master, self.gene_encoding, "Binary", "Real").grid(row=3, column=1)

        tk.Button(self.master, text="Next Iteration", command=self.next_iteration).grid(row=4, column=0, columnspan=2)

        self.fig = Figure(figsize=(5, 4), dpi=100)
        self.ax = self.fig.add_subplot(111)

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.master)
        self.canvas_widget = self.canvas.get_tk_widget()
        self.canvas_widget.grid(row=0, column=2, rowspan=5)

    def create_initial_population(self):
        x1_values = np.random.uniform(-5, 5, self.population_size.get())
        x2_values = np.random.uniform(-5, 5, self.population_size.get())

        self.population = np.column_stack((x1_values, x2_values))

    def update_plot(self):
        self.ax.clear()

        self.plot_function()
        x1_values = self.population[:, 0]
        x2_values = self.population[:, 1]
        self.ax.scatter(x1_values, x2_values, label="Population", color='white')

        if self.best_solution is not None:
            best_x1, best_x2 = self.best_solution
            self.ax.scatter(best_x1, best_x2, color='red', label="Best Solution")
            self.ax.text(best_x1, best_x2, f'({best_x1:.15f}, {best_x2:.15f})', color='red', fontsize=8, va='bottom')
            abs_min_text = f'Minimum: {self.function(best_x1, best_x2):.15f} at {best_x1:.15f}, {best_x2:.15f}'
            self.ax.text(0.5, -6, abs_min_text, ha='center', va='center', color='black', bbox=dict(facecolor='white', alpha=0.7))
        
        self.ax.text(0.5, 1.05, f'Iteration: {self.iteration}', transform=self.ax.transAxes, ha='center', va='center')

        self.ax.set_xlabel('x1')
        self.ax.set_ylabel('x2')
        self.ax.legend()

        self.canvas.draw()
    
    def plot_function(self):
        X1, X2 = np.meshgrid(self.x1_range, self.x2_range)
        Z = self.function(X1, X2)

        self.ax.contourf(X1, X2, Z, levels=100, cmap='viridis')

        self.canvas.draw()

    def next_iteration(self):
        self.iteration += 1

        self.mutation_rate = tk.IntVar(value=max(self.min_mutation_rate, self.mutation_rate.get() * self.mutation_decay_factor))

        selected_indices = self.selection()

        offspring = self.crossover(selected_indices)

        offspring = self.mutation(offspring)

        self.population = offspring

        self.best_solution = self.population[np.argmin(self.function(self.population[:, 0], self.population[:, 1]))]

        self.update_plot()
    
    def create_gene_modification_widgets(self):
        tk.Label(self.master, text="Individual Index:").grid(row=0, column=3, sticky="e")
        self.entry_index = tk.Entry(self.master)
        self.entry_index.grid(row=0, column=4)

        tk.Label(self.master, text="Gene 1:").grid(row=1, column=3, sticky="e")
        self.entry_gene1 = tk.Entry(self.master)
        self.entry_gene1.grid(row=1, column=4)

        tk.Label(self.master, text="Gene 2:").grid(row=2, column=3, sticky="e")
        self.entry_gene2 = tk.Entry(self.master)
        self.entry_gene2.grid(row=2, column=4)

        tk.Button(self.master, text="Apply Modification", command=self.apply_modification).grid(row=3, column=3, columnspan=2)

    def apply_modification(self):
        try:
            index = int(self.entry_index.get())
            new_gene1 = float(self.entry_gene1.get())
            new_gene2 = float(self.entry_gene2.get())

            self.population[index, 0] = new_gene1
            self.population[index, 1] = new_gene2

            self.update_plot()
        except ValueError:
            print("Invalid input. Please enter numeric values for gene modification.")

    def selection(self):
        tournament_size = min(5, self.population_size.get())
        selected_indices = []

        for _ in range(self.population_size.get()):
            tournament_indices = np.random.choice(self.population_size.get(), tournament_size, replace=False)
            tournament_fitness = self.function(self.population[tournament_indices, 0], self.population[tournament_indices, 1])
            winner_index = tournament_indices[np.argmin(tournament_fitness)]
            selected_indices.append(winner_index)

        return selected_indices

    def crossover(self, selected_indices):
        crossover_point = np.random.randint(1, self.population.shape[1])

        offspring = []
        for i in range(0, self.population_size.get(), 2):
            parent1 = self.population[selected_indices[i]]
            parent2 = self.population[selected_indices[i + 1]]

            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

            offspring.extend([child1, child2])

        return np.array(offspring)

    def mutation(self, offspring):
        mutation_mask = np.random.rand(*offspring.shape) < self.mutation_rate.get()

        if self.gene_encoding.get() == "Binary":
            offspring[mutation_mask] = 1 - offspring[mutation_mask]
        elif self.gene_encoding.get() == "Real":
            offspring[mutation_mask] += np.random.normal(scale=0.1, size=np.sum(mutation_mask))

        return offspring


In [33]:
if __name__ == "__main__":
    root = tk.Tk()
    app = GeneticAlgorithmApp(root)
    root.mainloop()